# Step 2 - Climate App

### I. First, we import the necessary libraries. This is what each library does:
* Numpy, Pandas - not used here. 
* datetime - allows python to read day, year, month and work with them together. 
* sqlalchemy - allows us to write SQL code inside of python. 
* automap_base - reads sql schema.
* session - accesses the database. 
* create_engine, func, desc - opens up the sql database. 
* Flask, jsonify - jsonify turns python dict into json object. 
* Request, Response - Carlos told the class via slack  to put this in. 

In [374]:
#import numpy as np
#import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, request
from werkzeug.wrappers import Request, Response

### II. Next, we link python to our database, make python classes, and start a session or tab in the browser. 

In [375]:
engine = create_engine('sqlite:///Resources/hawaii.sqlite', echo=False)

# Declare a Base using 'automap_base()'
Base = automap_base()

# Reflect Database into ORM classes
Base.prepare(engine, reflect=True)

# Save a reference to the measurenment table as 'Measurement'
Measurement = Base.classes.measurement

# Save a reference to the station table as 'Station'
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

### III. Next, we create the app which comes from the Flask library. 

In [376]:
app = Flask(__name__)

### IV. We create the home page using HTML. 
This page lists all available api routes.

In [377]:
@app.route("/")

def welcome():
   
    return (
        "<h1> Hawaii Precipitation and Weather Data</h1>"
        
        "<h3>Pick from the available routes below:</h3>"
        
        "Precipiation from 2016-08-23 to 2017-08-23.<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/precipitation'>Precipitation Data</a><br/><br/>"
        
        "A list of all the weather stations in Hawaii.<br/>"
        "<a target='_blank'href= 'http://localhost:9000/api/v1.0/stations'>Weather Stations Data</a><br/><br/>"
        
        "The Temperature Observations (tobs) from 2016-08-23 to 2017-08-23.<br/>"
        "<a target='_blank' href='http://localhost:9000/api/v1.0/tobs'>Temperature Observations Data</a><br/><br/>"
        
        "Type in a single date (i.e., 2017-02-24) to see the min, max and avg temperature since that date.<br/>"
        "<form action='http://localhost:9000/api/v1.0/temp/' method='get'><input name='start' id='start' value='' placeholder='yyyy-mm-dd' ><div><button>Enter</button></div></form>"
        
        "Type in the start and end dates (i.e., 2016-08-23 and 2017-08-23) to see the min, max and avg temperature for that range.<br/>"
        "<form action='http://localhost:9000/api/v1.0/temp-range/' method='get'> Start Date <input name='start' id='start' value='' placeholder='yyyy-mm-dd'>End Date <input name='end' id='end' value='' placeholder='yyyy-mm-dd'><div><button>Enter</button></div></form>"
        
        "<img src='http://www.kauai.com/blog/wp-content/uploads/2011/10/kauai-rainbow.jpg'>"
    )

### V. Query for the dates and temperature observations from the last year.
Converts the query results to a Dictionary using date as the 'key 'and 'tobs' as the value.


In [378]:
begin_date = dt.date(2017, 8, 23) - dt.timedelta(days=365)

@app.route("/api/v1.0/precipitation")

def precipitation():

    # Retrieve the last 12 months of precipitation data
    results = session.query(Measurement.date, Measurement.prcp).\
                        filter(Measurement.date > begin_date).\
                        order_by(Measurement.date).all()

    # Create a dictionary from the row data and append to a list of for the precipitation data
    precipitation_data = []
    for prcp_data in results:
        prcp_data_dict = {}
        prcp_data_dict["Date"] = prcp_data.date
        prcp_data_dict["Precipitation"] = prcp_data.prcp
        precipitation_data.append(prcp_data_dict)
        

    return jsonify(precipitation_data)


### VI. The following route creates a query and returns a json list of stations from the dataset. 

In [379]:
@app.route("/api/v1.0/stations")

def stations():
    
    # Query all the stations
    results = session.query(Station).all()

    # Create a dictionary from the row data and append to a list of all_stations.
    all_stations = []
    for stations in results:
        stations_dict = {}
        stations_dict["Station"] = stations.station
        stations_dict["Station Name"] = stations.name
        stations_dict["Latitude"] = stations.latitude
        stations_dict["Longitude"] = stations.longitude
        stations_dict["Elevation"] = stations.elevation
        all_stations.append(stations_dict)
    
    return jsonify(all_stations)

### VII. The following route returns a json list of Temperature Observations (tobs) for the previous year. 

In [380]:
@app.route("/api/v1.0/tobs")

def tobs():
    
    # Query all the stations and for the given date. 
    results = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
                    group_by(Measurement.date).\
                    filter(Measurement.date > begin_date).\
                    order_by(Measurement.station).all()
                    
    # Create a dictionary from the row data and append to a list of for the temperature data.
    temp_data = []
    for tobs_data in results:
        tobs_data_dict = {}
        tobs_data_dict["Station"] = tobs_data.station
        tobs_data_dict["Date"] = tobs_data.date
        tobs_data_dict["Temperature"] = tobs_data.tobs
        temp_data.append(tobs_data_dict)
    
    return jsonify(temp_data)
    

### VIII. The following route returns a json list of the minimum temperature, the average temperature, and the max temperature for a given start date. 
The 'request' library was imported in order to get the start dates from the url and create a search query of the start dates. 

In [381]:
@app.route("/api/v1.0/temp/", methods=['get'])

def start_stats(start=None):
    
    start = request.args.get('start')
    
    # Query all the stations and for the given date. 
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).all()

    # Create a dictionary from the row data and append to a list of for the temperature data.
    temp_stats = []
    
    for Tmin, Tmax, Tavg in results:
        temp_stats_dict = {}
        temp_stats_dict["Minimum Temp"] = Tmin
        temp_stats_dict["Maximum Temp"] = Tmax
        temp_stats_dict["Average Temp"] = Tavg
        temp_stats.append(temp_stats_dict)
    
    return jsonify(temp_stats)
    

###   IX. The following route returns a json list of the minimum temperature, the average temperature, and the max temperature for a given start-end date range. 
    

In [382]:
@app.route("/api/v1.0/temp-range/", methods=['get'])
def calc_stats(start=None, end=None):
    
    start = request.args.get('start')
    end   = request.args.get('end') 
    
    # Query all the stations and for the given range of dates. 
    results = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
    filter(Measurement.date >= start).filter(Measurement.date <= end).all()

    # Create a dictionary from the row data and append to a list of for the temperature data.
    begin_end_stats = []
    
    for Tmin, Tmax, Tavg in results:
        begin_end_stats_dict = {}
        begin_end_stats_dict["Minimum Temp"] = Tmin
        begin_end_stats_dict["Maximum Temp"] = Tmax
        begin_end_stats_dict["Average Temp"] = Tavg
        begin_end_stats_dict['start']=start
        begin_end_stats_dict['end']=end
        begin_end_stats.append(begin_end_stats_dict)
    
    return jsonify(begin_end_stats)

### X. The following command starts the application. To stop it, we must interrupt the code from running because "control+c" did not work for me. 

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Aug/2018 19:28:53] "GET / HTTP/1.1" 200 -


# To see the App, click on <a target='_blank' href='http://localhost:9000/' >this link</a> after running all the code by going to Cell -> Run All. 